In [5]:
import diplib as dip
import numpy as np
import os

# -------------------------------------------------------------------
# Configuration
# -------------------------------------------------------------------
image_dir = r"C:\Users\white\Music\IAM\A3\2025Assignment03Images"

# Part 3.1 files & known scale-bar length
scalebar_files = ["CamIm01.tif", "CamIm02.tif", "CamIm03.tif"]
known_length_um = 10.0  # the printed bar length in micrometers

# Part 3.2 filenames
tif_fname = "scale-img.tif"
ics_fname = "scale-img.ics"
ids_fname = "scale-img.ids"

# -------------------------------------------------------------------
# Part 3.1: Pixel-size calibration via fast top-hat (opening-subtract)
# -------------------------------------------------------------------
print("=== Part 3.1: Pixel-size calibration ===")
for fname in scalebar_files:
    path = os.path.join(image_dir, fname)
    if not os.path.exists(path):
        print(f"  ERROR: File not found: {path}")
        continue

    # 1) Read the TIFF
    img = dip.ImageReadTIFF(path)

    # 2) Estimate background by grey-value opening with an elliptical SE size [50,5]
    se_size = [50, 5]
    background = dip.Opening(img, se_size)

    # 3) Subtract to get “white top-hat”
    img_tophat = img - background

    # 4) Threshold (Otsu) to isolate the bar
    binary, thr = dip.Threshold(img_tophat)

    # 5) Measure pixel-length of the bar along X
    arr = np.asarray(binary, bool)
    cols = np.where(arr.any(axis=0))[0]
    pixel_length = int(cols.max() - cols.min())

    # 6) Compute pixel size [µm/px]
    pixel_size = known_length_um / pixel_length
    print(f"  {fname: <12} Pixels(bar)={pixel_length:<4}   Pixel size={pixel_size:.4f} µm/px")

# -------------------------------------------------------------------
# Part 3.2: Read & compare TIFF vs ICS
# -------------------------------------------------------------------
print("\n=== Part 3.2: Read & compare TIFF vs ICS ===")
tif_path = os.path.join(image_dir, tif_fname)
ics_path = os.path.join(image_dir, ics_fname)
ids_path = os.path.join(image_dir, ids_fname)

# 1) Dump the .ics header
if os.path.exists(ics_path):
    print("\n---- ICS header (scale-img.ics) ----")
    with open(ics_path, "r") as f:
        for line in f:
            print(line.rstrip())
else:
    print(f"  WARNING: ICS header not found: {ics_path}")

# 2) Load the TIFF
if not os.path.exists(tif_path):
    raise FileNotFoundError(f"TIFF not found: {tif_path}")
img_tif = dip.ImageReadTIFF(tif_path)

# 3) Load the ICS+IDS
if os.path.exists(ics_path) and os.path.exists(ids_path):
    img_ics = dip.ImageReadICS(ics_path)
else:
    img_ics = None
    print("  WARNING: ICS or IDS missing; skipping ICS load.")

# 4) Numeric comparison
arr_tif = np.asarray(img_tif)
print(f"\nTIFF shape: {arr_tif.shape}, dtype: {arr_tif.dtype}")
if img_ics is not None:
    arr_ics = np.asarray(img_ics)
    print(f"ICS shape:  {arr_ics.shape}, dtype: {arr_ics.dtype}")
    diff = arr_tif.astype(np.int32) - arr_ics.astype(np.int32)
    print("Difference stats (TIFF - ICS):",
          f"min={diff.min()}, max={diff.max()}, mean={diff.mean():.2f}, std={diff.std():.2f}")
else:
    print("Skipped numeric comparison due to missing ICS.")

# 5) File-size comparison
def file_size_mb(path):
    return os.path.getsize(path) / (1024*1024)

print("\nFile sizes:")
for fname in (tif_fname, ics_fname, ids_fname):
    p = os.path.join(image_dir, fname)
    if os.path.exists(p):
        print(f"  {fname}: {file_size_mb(p):.2f} MB")
    else:
        print(f"  {fname}: NOT FOUND")


=== Part 3.1: Pixel-size calibration ===
  CamIm01.tif  Pixels(bar)=2003   Pixel size=0.0050 µm/px
  CamIm02.tif  Pixels(bar)=2344   Pixel size=0.0043 µm/px
  CamIm03.tif  Pixels(bar)=2580   Pixel size=0.0039 µm/px

=== Part 3.2: Read & compare TIFF vs ICS ===

---- ICS header (scale-img.ics) ----

ics_version	1.0
filename	scale-img
layout	parameters	3
layout	order	bits	x	y
layout	sizes	32	256	256
layout	coordinates	video
layout	significant_bits	32
representation	format	real
representation	sign	signed
representation	byte_order	4	3	2	1
representation	SCIL_TYPE	f2d

TIFF shape: (256, 256), dtype: uint8
ICS shape:  (256, 256), dtype: float32
Difference stats (TIFF - ICS): min=0, max=0, mean=0.00, std=0.00

File sizes:
  scale-img.tif: 0.06 MB
  scale-img.ics: 0.00 MB
  scale-img.ids: 0.25 MB
